In [1]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import cobra
from cobra.test import create_test_model
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
model = create_test_model("textbook")
model.solver

a = cobra.io.read_sbml_model("iEC1344_C.xml")


Academic license - for non-commercial use only - expires 2024-03-07
Using license file /home/schwartzlab/gurobi.lic


In [2]:
a.reactions.EX_xyl__D_e.lower_bound= -10
a.reactions.EX_xyl__D_e.upper_bound= 1000

a.reactions.EX_glc__D_e.lower_bound= 0
a.reactions.EX_glc__D_e.upper_bound= 0


In [3]:
solution = a.optimize()
print(solution)

solution.objective_value


<Solution 0.806 at 0x7fccc43b50d0>


0.8061166555676651

In [ ]:
fmin = -1000
fmax = 1000
i = -1

while i< 2727:
    x = list()    # x and y need to be reinitialised for each new reaction
    y = list()
    f = fmin      # define a new variable to change the flux, fmin should remain a fixed value

    i = i + 1
    r = a.reactions[i]
    r_id = r.id

    foriginalmax = a.reactions.get_by_id(r_id).upper_bound
    foriginalmin = a.reactions.get_by_id(r_id).lower_bound
    a.reactions.get_by_id(r_id).lower_bound = a.reactions.get_by_id(r_id).upper_bound = f

    while f <= fmax:
        a.reactions.get_by_id(r_id).lower_bound = a.reactions.get_by_id(r_id).upper_bound = f
        
        solution = a.optimize()
        ov_new = solution.objective_value

        x.append(f)
        y.append(ov_new)

        f = f + 10

    d = pd.DataFrame(x, columns = ['flux'])
    d2 = pd.DataFrame(y, columns = ['Biomass'])
    d['Biomass'] = d2['Biomass']
    biomass_per_flux = d.to_csv('biomass_per_flux_for_reaction_{}_xylose.csv'.format(r_id), index = True)
    a.reactions.get_by_id(r_id).lower_bound = foriginalmin
    a.reactions.get_by_id(r_id).upper_bound = foriginalmax
    

/home/schwartzlab/miniconda3/lib/python3.8/site-packages/cobra/util/solver.py:508: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
